In [49]:
from waterlevels_oker import utils
from waterlevels_oker.data import get_okertal_training_data, get_schladen_training_data, get_ohrum_training_data, get_bridge_training_data, get_eisenbuettel_training_data, get_wendenwehr_training_data
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format = "retina"

In [177]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime
from datetime import timedelta
from waterlevels_oker import utils
from waterlevels_oker.config import *


def get_raw_weather_data(city: str) -> pd.DataFrame:
	"""
	Gets weather data from https://brightsky.dev/docs/#/operations/getWeather starting 2010-01-01 until 2024-07-31.

	Parameters
	----------
	city : str
		City from
		["okertal",
		"schladen",
		"ohrum",
		"schaeferbruecke",
		"eisenbuetteler_wehr",
		"wendenwehr"]

	Returns
	-------
	pd.Dataframe
		Raw weather data
	"""
	cities = {
		"okertal": [51.85, 10.45],
		"schladen": [52.019, 10.54],
		"ohrum": [52.11, 10.56],
		"schaeferbruecke": [52.17, 10.51],
		"eisenbuetteler_wehr": [52.24, 10.52],
		"wendenwehr": [52.27, 10.52],
	}

	weather_raw = pd.DataFrame()
	for year in range(2012, 2025):
		params = {
			"date": "{}-01-01".format(year),
			"last_date": "{}-07-31".format(year + 1),
			"lat": cities[city][0],
			"lon": cities[city][1],
		}

		url = "https://api.brightsky.dev/weather"
		response = requests.get(url, params=params)
		weather = pd.DataFrame(response.json()["weather"])
		weather_raw = pd.concat([weather_raw, weather])

	return weather_raw


def get_forecast(city: str) -> pd.DataFrame:
	"""
	Get weather forecaste from start to end date for the city specified.

	city : str
		City from
		["okertal",
		"schladen",
		"ohrum",
		"schaeferbruecke",
		"eisenbuetteler_wehr",
		"wendenwehr"]

	Returns
	-------
	pd.DataFrame
		Dataframe containing the weather forecast
	"""

	cities = {
		"okertal": [51.85, 10.45],
		"schladen": [52.019, 10.54],
		"ohrum": [52.11, 10.56],
		"schaeferbruecke": [52.17, 10.51],
		"eisenbuetteler_wehr": [52.24, 10.52],
		"wendenwehr": [52.27, 10.52],
	}

	start_date = datetime(2023, 12, 21)
	end_date = start_date + timedelta(days=9)

	params = {
		"date": start_date.isoformat(),
		"last_date": end_date.isoformat(),
		"lat": cities[city][0],
		"lon": cities[city][1],
	}

	url = "https://api.brightsky.dev/weather"
	response = requests.get(url, params=params)

	forecast = pd.DataFrame(response.json()["weather"])

	preprocessed = preprocess_weather_data(forecast)

	return preprocessed


def preprocess_weather_data(raw_data) -> pd.DataFrame:
	"""
	Preprocesses raw weather data.

	Parameters
	----------
	raw_data

	Returns
	-------
	pd.DataFrame
		Processed weather data
	"""

	raw_data = raw_data.assign(
		timestamp=pd.to_datetime(raw_data["timestamp"])
	).set_index("timestamp")

	# Drop timezone from datetime index
	raw_data.index = raw_data.index.tz_localize(None)

	drop_cols = [
		"source_id",
		"condition",
		"precipitation_probability",
		"precipitation_probability_6h",
		# "fallback_source_ids",
		"icon",
	]
	raw_data = raw_data.drop(columns=drop_cols)
	if "fallback_source_ids" in raw_data.columns:
		raw_data = raw_data.drop(columns="fallback_source_ids")

	# Impute missing sunshine values during nighttime with 0
	night_missing_sunshine = raw_data.loc[
		raw_data["sunshine"].isna() & raw_data.index.hour.isin([21, 22, 23, 0, 1, 2])
	].index
	raw_data.loc[night_missing_sunshine, "sunshine"] = 0

	raw_data = raw_data.bfill()

	daily_weather_data = raw_data.groupby(raw_data.index.date).mean()

	return daily_weather_data


def preprocess_okertal_data():
	okertal_data = pd.read_excel(utils.get_raw_path("Oker Daten.xlsx"), index_col=0)

	okertal_data = okertal_data.loc[:, ["Stauinhalt Okertalsperre [Mio.m³]"]].rename(
		columns={
			"Stauinhalt Okertalsperre [Mio.m³]": "fill_[mio.m³]",
		}
	)

	return okertal_data


def get_okertal_training_data():
	weather = get_raw_weather_data("okertal")

	processed_weather = preprocess_weather_data(weather)

	okertal_data = preprocess_okertal_data()

	full_data = processed_weather.join(okertal_data, how="inner").reset_index(
		names=["timestamp"]
	)
	return full_data


def get_schladen_training_data():
	weather = get_raw_weather_data("schladen")

	processed_weather = preprocess_weather_data(weather)

	okertal_data = preprocess_okertal_data()
	schladen_data = preprocess_brunswick_data().loc[:, "schladen"]

	full_data = (
		processed_weather.join(okertal_data, how="inner")
		.join(schladen_data, how="inner")
		.reset_index(names=["timestamp"])
	)
	return full_data


def get_ohrum_training_data():
	weather = get_raw_weather_data("ohrum")

	processed_weather = preprocess_weather_data(weather)

	schladen_data = preprocess_brunswick_data().loc[:, "schladen"]
	ohrum_data = preprocess_brunswick_data().loc[:, "ohrum_level"]

	full_data = (
		processed_weather.join(ohrum_data, how="inner")
		.join(schladen_data, how="inner")
		.reset_index(names=["timestamp"])
	)
	return full_data


def get_bridge_training_data():
	weather = get_raw_weather_data("schaeferbruecke")

	processed_weather = preprocess_weather_data(weather)

	ohrum_data = preprocess_brunswick_data().loc[:, "ohrum_level"]
	bridge_data = preprocess_brunswick_data().loc[:, "schaeferbridge"]

	full_data = (
		processed_weather.join(ohrum_data, how="inner")
		.join(bridge_data, how="inner")
		.reset_index(names=["timestamp"])
	)
	return full_data


def get_eisenbuettel_training_data():
	weather = get_raw_weather_data("eisenbuetteler_wehr")

	processed_weather = preprocess_weather_data(weather)

	bridge_data = preprocess_brunswick_data().loc[:, "schaeferbridge"]
	eisenbütteler_wehr_data = preprocess_brunswick_data().loc[:, "eisenbuetteler_wehr"]

	full_data = (
		processed_weather.join(bridge_data, how="inner")
		.join(eisenbütteler_wehr_data, how="inner")
		.reset_index(names=["timestamp"])
	)
	return full_data


def get_wendenwehr_training_data():
	weather = get_raw_weather_data("wendenwehr")

	processed_weather = preprocess_weather_data(weather)
	eisenbütteler_wehr_data = preprocess_brunswick_data().loc[:, "eisenbuetteler_wehr"]
	wendenwehr_data = preprocess_brunswick_data().loc[:, "wendenwehr"]

	full_data = (
		processed_weather.join(eisenbütteler_wehr_data, how="inner")
		.join(wendenwehr_data, how="inner")
		.reset_index(names=["timestamp"])
	)
	return full_data


def get_okertal_forecast_data():
	weather = get_forecast("okertal")

	processed_weather = preprocess_weather_data(weather)

	return processed_weather


def preprocess_brunswick_data() -> pd.DataFrame:
 	measurements_2019 = pd.read_excel(
 		utils.get_raw_path("Übersicht Pegelwerte 2019-2023.xlsx"),
 		sheet_name="2019",
 		header=[0, 1],
 	)

 	measurements_2019.columns = measurements_2019.columns.to_flat_index()
 	measurements_2019 = measurements_2019.rename(
 		columns={("Datum", "Unnamed: 1_level_1"): "timestamp"}
 	)

 	measurements_2020 = pd.read_excel(
 		utils.get_raw_path("Übersicht Pegelwerte 2019-2023.xlsx"),
 		sheet_name="2020",
 		header=[0, 1],
 	)

 	measurements_2020.columns = measurements_2020.columns.to_flat_index()
 	measurements_2020 = measurements_2020.rename(
 		columns={("2020", "Unnamed: 1_level_1"): "timestamp"}
 	)

 	measurements_2021 = pd.read_excel(
 		utils.get_raw_path("Übersicht Pegelwerte 2019-2023.xlsx"),
 		sheet_name="2021",
 		header=[1, 2],
 	)

 	measurements_2021.columns = measurements_2021.columns.to_flat_index()
 	measurements_2021 = measurements_2021.rename(
 		columns={("2021", "Unnamed: 1_level_1"): "timestamp"}
 	)

 	measurements_2022 = pd.read_excel(
 		utils.get_raw_path("Übersicht Pegelwerte 2019-2023.xlsx"),
 		sheet_name="2022",
 		header=[0, 1],
 	)

 	measurements_2022.columns = measurements_2022.columns.to_flat_index()
 	measurements_2022 = measurements_2022.rename(
 		columns={(2022, "Uhrzeit.2"): "timestamp"}
 	)

 	measurements_2023 = pd.read_excel(
 		utils.get_raw_path("Übersicht Pegelwerte 2019-2023.xlsx"),
 		sheet_name="2023",
 		header=[0, 1],
 	)

 	measurements_2023.columns = measurements_2023.columns.to_flat_index()
 	measurements_2023 = measurements_2023.rename(
 		columns={
 			(2023, "Uhrzeit.2"): "timestamp",
 		}
 	)

 	all_measurements = pd.concat(
 		[
 			measurements_2019,
 			measurements_2020,
 			measurements_2021,
 			measurements_2022,
 			measurements_2023,
 		]
 	).set_index("timestamp")

 	all_measurements = all_measurements[
 		[
 			("Schladen", 88.72),
 			("Ohrum", 75.54),
 			("sensoweb", "Schäferbr."),
 			("Eisenbütteler Wehr", "OW"),
 			("Petriwehr", "OW"),
 			("Wendenwehr", "OW"),
 		]
 	]

 	all_measurements = all_measurements.rename(
 		columns={
 			("Schladen", 88.72): "schladen",
 			("Ohrum", 75.54): "ohrum_level",
 			("sensoweb", "Schäferbr."): "schaeferbridge",
 			("Eisenbütteler Wehr", "OW"): "eisenbuetteler_wehr",
 			("Petriwehr", "OW"): "petriwehr",
 			("Wendenwehr", "OW"): "wendenwehr",
 		}
 	)

 	all_measurements = all_measurements.replace("-", np.nan).bfill()

 	return all_measurements

In [51]:
df_okertal = get_okertal_training_data()
df_schladen = get_schladen_training_data()
df_ohrum = get_ohrum_training_data()
df_bridge = get_bridge_training_data()
df_eisenbuettel = get_eisenbuettel_training_data()
df_wendenwehr = get_wendenwehr_training_data()

/var/folders/9q/rv1kd_ws7x7_tybjxx3921bm0000gn/T/ipykernel_38749/2847495355.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_raw = pd.concat([weather_raw, weather])
/var/folders/9q/rv1kd_ws7x7_tybjxx3921bm0000gn/T/ipykernel_38749/2847495355.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_raw = pd.concat([weather_raw, weather])
/var/folders/9q/rv1kd_ws7x7_tybjxx3921bm0000gn/T/ipykernel_38749/2847495355.py:344: FutureWarning: Downcasting behavior in `replace` is deprecated

In [79]:
df_okertal

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,fill_[mio.m³]
0,2012-10-01,0.000000,1017.879167,24.125000,13.570833,253.333333,6.762500,32.541667,5.558333,59.708333,20791.666667,250.416667,13.416667,0.154250,13.448
1,2012-10-02,0.000000,1016.058333,16.291667,15.437500,227.916667,10.433333,79.416667,8.245833,62.291667,26000.000000,228.333333,20.091667,0.135625,13.323
2,2012-10-03,0.012500,1012.350000,4.958333,15.195833,210.416667,16.404167,93.583333,9.437500,68.666667,31500.000000,209.583333,34.133333,0.037042,13.215
3,2012-10-04,0.554167,1012.075000,0.000000,12.037500,253.333333,14.416667,96.750000,9.041667,82.041667,18666.666667,264.166667,37.883333,0.003750,13.165
4,2012-10-05,0.133333,1009.641667,0.041667,12.445833,224.583333,18.700000,92.958333,8.879167,78.916667,30041.666667,218.333333,36.083333,0.017833,13.253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,2024-07-27,1.225000,1013.991667,1.229167,19.212500,150.416667,8.685417,92.687500,17.041667,87.791667,17311.875000,163.750000,15.006250,0.132042,29.868
4318,2024-07-28,0.112500,1022.374167,19.500000,17.933333,278.125000,11.289583,52.625000,13.575000,77.708333,41243.541667,285.000000,22.902083,0.249583,29.968
4319,2024-07-29,0.000000,1024.551667,34.833333,17.766667,137.083333,3.212500,3.083333,10.491667,66.083333,52706.666667,135.416667,8.187500,0.309667,30.017
4320,2024-07-30,0.000000,1017.787500,34.166667,21.587500,173.333333,6.700000,18.625000,12.462500,59.166667,62237.916667,186.250000,13.179167,0.290458,30.025


In [158]:
filtered_df = df_okertal[(df_okertal["timestamp"] >= "2023-12-21") & (df_okertal["timestamp"] <= "2023-12-29")]
filtered_df

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,fill_[mio.m³]
4098,2023-12-21,0.625000,993.045833,0.000000,7.433333,263.750000,31.120833,92.000000,4.191667,79.916667,30437.500000,265.416667,62.758333,0.0,40.483
4099,2023-12-22,0.337500,995.725000,0.125000,4.454167,291.250000,30.408333,94.041667,0.608333,76.500000,33434.166667,295.000000,63.583333,0.0,41.678
4100,2023-12-23,1.079167,1003.070833,0.000000,4.454167,284.166667,19.170833,93.000000,2.225000,85.458333,20193.333333,283.333333,39.350000,0.0,41.765
4101,2023-12-24,0.233333,1000.654167,1.666667,10.337500,255.000000,30.616667,95.666667,7.237500,81.083333,48709.166667,255.000000,62.254167,0.0,43.181
4102,2023-12-25,0.300000,1003.966667,5.666667,10.429167,262.916667,25.429167,95.708333,7.350000,81.208333,34339.583333,264.583333,50.050000,0.0,46.020
4103,2023-12-26,0.241667,1009.987500,9.791667,8.691667,271.666667,29.600000,77.916667,4.050000,72.958333,49945.833333,272.916667,58.279167,0.0,47.209
4104,2023-12-27,0.000000,1018.783333,1.333333,5.783333,180.416667,9.850000,77.541667,1.350000,74.375000,46632.083333,189.166667,23.870833,0.0,47.153
4105,2023-12-28,0.000000,1011.662500,7.291667,8.929167,206.666667,16.304167,86.916667,3.304167,68.083333,67325.416667,212.916667,41.383333,0.0,46.513
4106,2023-12-29,0.383333,1007.300000,1.666667,8.741667,241.666667,28.741667,79.375000,4.904167,76.833333,49774.166667,245.416667,61.425000,0.0,45.392


In [80]:
y_okertal = df_okertal["fill_[mio.m³]"]
X_okertal = df_okertal.drop(columns=["fill_[mio.m³]","timestamp"])

In [81]:
X_train_okertal, X_test_okertal, y_train_okertal, y_test_okertal = train_test_split(X_okertal, y_okertal, test_size=0.2, random_state=42)

In [82]:
# Scaling the data
scaler1 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_okertal)
X_test_scaled = scaler1.transform(X_test_okertal)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_okertal)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_okertal, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

Random Forest: RMSE = 6.977022109125862


In [83]:
import pickle

pickle_file = "model_okertal.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

In [84]:
# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_okertal, predictions))
print(rmse)


6.977022109125862


In [85]:
df_schladen

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,fill_[mio.m³],schladen
0,2019-01-01,0.200000,1022.237083,0.583333,5.766667,281.250000,24.575000,92.416667,2.616667,80.291667,35283.750000,281.666667,51.612500,0.058,17.441,89.44
1,2019-01-02,0.004167,1029.539167,4.000000,1.637500,310.416667,22.908333,60.791667,-3.229167,70.583333,48642.500000,315.000000,43.266667,0.058,17.780,89.45
2,2019-01-03,0.000000,1035.944167,15.166667,0.129167,303.333333,12.225000,45.791667,-4.379167,72.208333,47961.666667,302.500000,23.325000,0.058,17.989,89.44
3,2019-01-04,0.058333,1030.626250,0.000000,2.312500,292.083333,18.408333,97.291667,-0.304167,83.750000,48196.250000,290.833333,38.604167,0.058,18.138,89.44
4,2019-01-05,0.341667,1023.303750,0.000000,6.000000,271.666667,20.629167,95.125000,5.500000,96.625000,12731.666667,275.000000,40.495833,0.058,18.409,89.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,0.000000,1018.783333,1.333333,4.395833,180.416667,9.850000,77.541667,1.054167,78.916667,46632.083333,189.166667,23.870833,0.002,47.153,91.57
1091,2023-12-28,0.000000,1011.662500,7.291667,8.983333,206.666667,16.304167,86.916667,2.750000,65.166667,67325.416667,212.916667,41.383333,0.002,46.513,91.29
1092,2023-12-29,0.191667,1007.300000,1.666667,8.775000,241.666667,28.741667,79.375000,4.070833,72.458333,49774.166667,245.416667,61.425000,0.002,45.392,91.24
1093,2023-12-30,0.195833,1011.570833,10.500000,6.845833,230.416667,19.458333,94.083333,2.104167,71.958333,54641.666667,234.583333,41.795833,0.002,44.588,91.31


In [86]:
y_schladen = df_schladen["schladen"]
X_schladen = df_schladen.drop(columns=["schladen","timestamp"])

In [87]:
X_train_schladen, X_test_schladen, y_train_schladen, y_test_schladen = train_test_split(X_schladen, y_schladen, test_size=0.2, random_state=42)

In [88]:
# Scaling the data
scaler2 = StandardScaler()
X_train_scaled = scaler2.fit_transform(X_train_schladen)
X_test_scaled = scaler2.transform(X_test_schladen)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_schladen)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_schladen, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

Random Forest: RMSE = 0.056165301748934734


In [89]:
import pickle

pickle_file = "model_schladen.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

In [90]:
# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_schladen, predictions))
print(rmse)


0.056165301748934734


In [91]:
df_ohrum

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,ohrum_level,schladen
0,2019-01-01,0.200000,1022.125000,1.000000,6.012500,279.166667,27.120833,75.666667,3.245833,82.541667,19096.250000,282.500000,48.912500,0.015208,76.83,89.44
1,2019-01-02,0.004167,1029.836667,13.041667,2.229167,305.416667,20.704167,27.916667,-2.866667,69.666667,43170.833333,297.083333,33.554167,0.043208,76.90,89.45
2,2019-01-03,0.000000,1035.993750,7.791667,-0.391667,270.416667,9.279167,51.541667,-2.566667,85.416667,30382.500000,271.666667,14.775000,0.034583,76.76,89.44
3,2019-01-04,0.058333,1030.552917,0.000000,1.937500,268.333333,18.804167,95.666667,0.866667,92.583333,7930.416667,265.416667,31.800000,0.007333,76.73,89.44
4,2019-01-05,0.341667,1023.448333,0.000000,6.358333,271.666667,19.612500,96.208333,5.666667,95.416667,20146.250000,268.750000,31.945833,0.013167,76.85,89.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,0.000000,1018.058333,0.000000,4.479167,182.500000,12.654167,54.958333,2.112500,85.041667,49354.166667,185.000000,22.287500,0.018708,79.87,91.57
1091,2023-12-28,0.000000,1010.620833,5.333333,9.316667,225.833333,19.783333,68.375000,4.058333,69.791667,71236.250000,223.750000,37.275000,0.031625,79.83,91.29
1092,2023-12-29,0.191667,1006.341667,0.000000,9.204167,233.750000,21.345833,69.458333,5.662500,78.833333,62213.750000,231.250000,41.150000,0.010708,79.75,91.24
1093,2023-12-30,0.195833,1010.833333,5.375000,7.212500,241.666667,16.479167,48.291667,3.483333,77.166667,59780.000000,239.583333,30.979167,0.034833,79.71,91.31


In [92]:
y_ohrum = df_ohrum["ohrum_level"]
X_ohrum = df_ohrum.drop(columns=["ohrum_level","timestamp"])

In [93]:
X_train_ohrum, X_test_ohrum, y_train_ohrum, y_test_ohrum = train_test_split(X_ohrum, y_ohrum, test_size=0.2, random_state=42)

In [94]:
# Scaling the data
scaler3 = StandardScaler()
X_train_scaled = scaler3.fit_transform(X_train_ohrum)
X_test_scaled = scaler3.transform(X_test_ohrum)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_ohrum)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_ohrum, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

Random Forest: RMSE = 0.07463551732025471


In [95]:
import pickle

pickle_file = "model_ohrum.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

In [96]:
# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_ohrum, predictions))
print(rmse)


0.07463551732025471


In [97]:
df_bridge

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,ohrum_level,schaeferbridge
0,2019-01-01,0.058333,1021.962500,0.875000,6.250000,277.916667,28.220833,86.166667,3.387500,82.083333,21237.083333,279.583333,51.179167,0.015208,76.83,72.45
1,2019-01-02,0.000000,1029.574167,13.041667,2.420833,308.333333,21.941667,37.333333,-2.795833,69.041667,43075.416667,300.000000,35.795833,0.043208,76.90,72.51
2,2019-01-03,0.000000,1036.089167,7.791667,-0.287500,275.833333,9.320833,51.541667,-2.504167,85.166667,32230.000000,277.500000,14.866667,0.034583,76.76,72.46
3,2019-01-04,0.004167,1031.060000,0.000000,2.587500,270.000000,17.945833,94.041667,1.558333,92.750000,9067.083333,265.416667,30.662500,0.007333,76.73,72.43
4,2019-01-05,0.158333,1023.469583,0.000000,6.958333,305.416667,20.158333,94.041667,6.120833,94.375000,21300.833333,300.833333,33.300000,0.013167,76.85,72.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,0.000000,1018.058333,0.000000,4.479167,182.500000,12.654167,54.958333,2.112500,85.041667,49354.166667,185.000000,22.287500,0.018708,79.87,75.22
1091,2023-12-28,0.000000,1010.620833,5.333333,9.316667,225.833333,19.783333,68.375000,4.058333,69.791667,71236.250000,223.750000,37.275000,0.031625,79.83,75.21
1092,2023-12-29,0.191667,1006.341667,0.000000,9.204167,233.750000,21.345833,69.458333,5.662500,78.833333,62213.750000,231.250000,41.150000,0.010708,79.75,75.15
1093,2023-12-30,0.029167,1010.833333,5.375000,7.212500,241.666667,16.479167,48.291667,3.483333,77.166667,59780.000000,239.583333,30.979167,0.034833,79.71,75.13


In [98]:
y_bridge = df_bridge["schaeferbridge"]
X_bridge = df_bridge.drop(columns=["schaeferbridge","timestamp"])


X_train_bridge, X_test_bridge, y_train_bridge, y_test_bridge = train_test_split(X_bridge, y_bridge, test_size=0.2, random_state=42)

# Scaling the data
scaler4 = StandardScaler()
X_train_scaled = scaler4.fit_transform(X_train_bridge)
X_test_scaled = scaler4.transform(X_test_bridge)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_bridge)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_bridge, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

import pickle

pickle_file = "model_bridge.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_bridge, predictions))
print(rmse)


Random Forest: RMSE = 0.10857646948592334
0.10857646948592334


In [99]:
df_eisenbuettel

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,schaeferbridge,eisenbuetteler_wehr
0,2019-01-01,0.058333,1021.962500,0.875000,6.250000,277.916667,28.220833,86.166667,3.387500,82.083333,21237.083333,279.583333,51.179167,0.015208,72.45,70.75
1,2019-01-02,0.000000,1029.574167,13.041667,2.420833,308.333333,21.941667,37.333333,-2.795833,69.041667,43075.416667,300.000000,35.795833,0.043208,72.51,70.75
2,2019-01-03,0.000000,1036.089167,7.791667,-0.287500,275.833333,9.320833,51.541667,-2.504167,85.166667,32230.000000,277.500000,14.866667,0.034583,72.46,70.73
3,2019-01-04,0.004167,1031.060000,0.000000,2.587500,270.000000,17.945833,94.041667,1.558333,92.750000,9067.083333,265.416667,30.662500,0.007333,72.43,70.75
4,2019-01-05,0.158333,1023.469583,0.000000,6.958333,305.416667,20.158333,94.041667,6.120833,94.375000,21300.833333,300.833333,33.300000,0.013167,72.43,70.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,0.000000,1018.058333,0.000000,4.479167,182.500000,12.654167,54.958333,2.112500,85.041667,49354.166667,185.000000,22.287500,0.018708,75.22,71.06
1091,2023-12-28,0.000000,1010.620833,5.333333,9.316667,225.833333,19.783333,68.375000,4.058333,69.791667,71236.250000,223.750000,37.275000,0.031625,75.21,71.07
1092,2023-12-29,0.191667,1006.341667,0.000000,9.204167,233.750000,21.345833,69.458333,5.662500,78.833333,62213.750000,231.250000,41.150000,0.010708,75.15,70.72
1093,2023-12-30,0.029167,1010.833333,5.375000,7.212500,241.666667,16.479167,48.291667,3.483333,77.166667,59780.000000,239.583333,30.979167,0.034833,75.13,70.60


In [100]:
y_eisenbuettel = df_eisenbuettel["eisenbuetteler_wehr"]
X_eisenbuettel = df_eisenbuettel.drop(columns=["eisenbuetteler_wehr","timestamp"])


X_train_eisenbuettel, X_test_eisenbuettel, y_train_eisenbuettel, y_test_eisenbuettel = train_test_split(X_eisenbuettel, y_eisenbuettel, test_size=0.2, random_state=42)

# Scaling the data
scaler5 = StandardScaler()
X_train_scaled = scaler5.fit_transform(X_train_eisenbuettel)
X_test_scaled = scaler5.transform(X_test_eisenbuettel)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_eisenbuettel)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_eisenbuettel, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

import pickle

pickle_file = "model_eisenbuettel.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_eisenbuettel, predictions))
print(rmse)


Random Forest: RMSE = 0.18114436257119362
0.18114436257119362


In [101]:
df_wendenwehr

,timestamp,precipitation,pressure_msl,sunshine,temperature,wind_direction,wind_speed,cloud_cover,dew_point,relative_humidity,visibility,wind_gust_direction,wind_gust_speed,solar,eisenbuetteler_wehr,wendenwehr
0,2019-01-01,0.058333,1021.962500,0.875000,6.250000,277.916667,28.220833,86.166667,3.387500,82.083333,21237.083333,279.583333,51.179167,0.015208,70.75,69.24
1,2019-01-02,0.000000,1029.574167,13.041667,2.420833,308.333333,21.941667,37.333333,-2.795833,69.041667,43075.416667,300.000000,35.795833,0.043208,70.75,69.27
2,2019-01-03,0.000000,1036.089167,7.791667,-0.287500,275.833333,9.320833,51.541667,-2.504167,85.166667,32230.000000,277.500000,14.866667,0.034583,70.73,69.33
3,2019-01-04,0.004167,1031.060000,0.000000,2.587500,270.000000,17.945833,94.041667,1.558333,92.750000,9067.083333,265.416667,30.662500,0.007333,70.75,69.22
4,2019-01-05,0.158333,1023.469583,0.000000,6.958333,305.416667,20.158333,94.041667,6.120833,94.375000,21300.833333,300.833333,33.300000,0.013167,70.75,69.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,0.000000,1018.058333,0.000000,4.479167,182.500000,12.654167,54.958333,2.112500,85.041667,49354.166667,185.000000,22.287500,0.018708,71.06,68.75
1091,2023-12-28,0.000000,1010.620833,5.333333,9.316667,225.833333,19.783333,68.375000,4.058333,69.791667,71236.250000,223.750000,37.275000,0.031625,71.07,68.67
1092,2023-12-29,0.191667,1006.341667,0.000000,9.204167,233.750000,21.345833,69.458333,5.662500,78.833333,62213.750000,231.250000,41.150000,0.010708,70.72,68.42
1093,2023-12-30,0.029167,1010.833333,5.375000,7.212500,241.666667,16.479167,48.291667,3.483333,77.166667,59780.000000,239.583333,30.979167,0.034833,70.60,68.27


In [102]:
y_wendenwehr = df_wendenwehr["wendenwehr"]
X_wendenwehr = df_wendenwehr.drop(columns=["wendenwehr","timestamp"])


X_train_wendenwehr, X_test_wendenwehr, y_train_wendenwehr, y_test_wendenwehr = train_test_split(X_wendenwehr, y_wendenwehr, test_size=0.2, random_state=42)

# Scaling the data
scaler6 = StandardScaler()
X_train_scaled = scaler6.fit_transform(X_train_wendenwehr)
X_test_scaled = scaler6.transform(X_test_wendenwehr)

# Define models
models = {
    'Random Forest': RandomForestRegressor()
}

# Train and evaluate models
rmse_results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train_wendenwehr)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_wendenwehr, y_pred))
    rmse_results[name] = rmse

# Display the RMSE results
for name, rmse in rmse_results.items():
    print(f'{name}: RMSE = {rmse}')

import pickle

pickle_file = "model_wendenwehr.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

# Opening a pickel file

with open(pickle_file, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_wendenwehr, predictions))
print(rmse)


Random Forest: RMSE = 0.16565379850429574
0.16565379850429574


Testing Pipeline

In [146]:
weather_data_okertal = get_forecast("okertal")
# weather_data_schladen = get_forecast("schladen")
# weather_data_ohrum = get_forecast("ohrum")
# weather_data_bridge = get_forecast("schaeferbruecke")
# weather_data_eisenbuettel = get_forecast("eisenbuetteler_wehr")
# weather_data_wendenwehr = get_forecast("wendenwehr")

In [179]:
import pickle


def predict_okertal(weather_data_okertal):
    # weather_data_okertal.reset_index(inplace=True)
    # print(weather_data_okertal)
    # weather_data_okertal = weather_data_okertal.drop(columns=["timestamp"])

    pickle_file = "model_okertal.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler1.transform(weather_data_okertal)
    fill = loaded_model.predict(scaled)

    return fill


def predict_schladen(fill,weather_data_schladen):

    # weather_data_schladen.reset_index(inplace=True)
    # weather_data_schladen = weather_data_schladen.drop(columns=["timestamp"])
    fill_df = pd.DataFrame(fill, index=weather_data_schladen.index, columns=['fill_[mio.m³]'])
    combined_df = weather_data_schladen.join(fill_df, how="inner")

    pickle_file = "model_schladen.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler2.transform(combined_df)

    schladen = loaded_model.predict(scaled)
    return schladen

def predict_ohrum(schladen, weather_data_ohrum):

    # weather_data_ohrum.reset_index(inplace=True)
    # weather_data_ohrum = weather_data_ohrum.drop(columns=["timestamp"])

    fill_df = pd.DataFrame(schladen, index=weather_data_ohrum.index, columns=['schladen'])
    combined_df = weather_data_ohrum.join(fill_df, how="inner")

    pickle_file = "model_ohrum.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler3.transform(combined_df)
    ohrum = loaded_model.predict(scaled)
    return ohrum

def predict_bridge(ohrum, weather_data_bridge):

    # weather_data_bridge.reset_index(inplace=True)
    # weather_data_bridge = weather_data_bridge.drop(columns=["timestamp"])

    fill_df = pd.DataFrame(ohrum, index=weather_data_bridge.index, columns=['ohrum_level'])
    combined_df = weather_data_bridge.join(fill_df, how="inner")

    pickle_file = "model_bridge.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler4.transform(combined_df)
    bridge = loaded_model.predict(scaled)

    return bridge

def predict_eisenbuettel(bridge, weather_data_eisenbuettel):

    # weather_data_eisenbuettel.reset_index(inplace=True)
    # weather_data_eisenbuettel = weather_data_eisenbuettel.drop(columns=["timestamp"])
    fill_df = pd.DataFrame(bridge, index=weather_data_eisenbuettel.index, columns=['schaeferbridge'])
    combined_df = weather_data_eisenbuettel.join(fill_df, how="inner")

    pickle_file = "model_eisenbuettel.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler5.transform(combined_df)
    eisenbuettel = loaded_model.predict(scaled)

    return eisenbuettel

def predict_wendenwehr(eisenbuettel, weather_data_wendenwehr):

    # weather_data_wendenwehr.reset_index(inplace=True)
    # weather_data_wendenwehr = weather_data_wendenwehr.drop(columns=["timestamp"])
    fill_df = pd.DataFrame(eisenbuettel, index=weather_data_wendenwehr.index, columns=['eisenbuetteler_wehr'])
    combined_df = weather_data_wendenwehr.join(fill_df, how="inner")

    pickle_file = "model_wendenwehr.pkl"
    with open(pickle_file, 'rb') as file:
        loaded_model = pickle.load(file)

    scaled = scaler6.transform(combined_df)
    wendenwehr = loaded_model.predict(scaled)

    return wendenwehr


In [169]:
def forecast():
	# okertal
	okertal_weather = get_forecast("okertal")
	fill = predict_okertal(okertal_weather)

	# schladen
	schladen_weather = get_forecast("schladen")
	schladen = predict_schladen(fill, schladen_weather)

	# ohrum
	ohrum_weather = get_forecast("ohrum")
	ohrum = predict_ohrum(schladen, ohrum_weather)

	# schaeferbruecke
	schaeferbruecke_weather = get_forecast("schaeferbruecke")
	schaeferbruecke = predict_bridge(ohrum, schaeferbruecke_weather)

	# eisenbuetteler_wehr
	eisenbuetteler_wehr_weather = get_forecast("eisenbuetteler_wehr")
	eisenbuetteler_wehr = predict_eisenbuettel(schaeferbruecke, eisenbuetteler_wehr_weather)

	# wendenwehr
	wendenwehr_weather = get_forecast("wendenwehr")
	wendenwehr = predict_wendenwehr(eisenbuetteler_wehr, wendenwehr_weather)

	return wendenwehr

In [226]:
okertal_weather = get_forecast("okertal")
fill = predict_okertal(okertal_weather)
fill

array([27.51748, 29.80684, 25.99978, 28.07637, 28.35667, 27.83294,
       29.10294, 27.55397, 30.10101, 30.43844])

In [227]:
filtered_df = df_okertal[(df_okertal["timestamp"] >= "2023-12-21") & (df_okertal["timestamp"] <= "2023-12-30")]
filtered_df["fill_[mio.m³]"]

4098    40.483
4099    41.678
4100    41.765
4101    43.181
4102    46.020
4103    47.209
4104    47.153
4105    46.513
4106    45.392
4107    44.588
Name: fill_[mio.m³], dtype: float64

In [228]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["fill_[mio.m³]"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=fill, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["fill_[mio.m³]"].min(), fill.min()))
y_max = np.ceil(max(filtered_df["fill_[mio.m³]"].max(), fill.max()))

fig.update_layout(
    title="Okertal Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 10, y_max + 10],
        dtick=5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [229]:
schladen_weather = get_forecast("schladen")
schladen = predict_schladen(fill, schladen_weather)
schladen

array([89.678  , 89.7431 , 89.5506 , 89.5798 , 89.5827 , 89.5748 ,
       89.4879 , 89.53073, 89.5926 , 89.6372 ])

In [230]:
filtered_df = df_schladen[(df_schladen["timestamp"] >= "2023-12-21") & (df_schladen["timestamp"] <= "2023-12-30")]
filtered_df["schladen"]

1084    90.12
1085    90.75
1086    90.94
1087    91.48
1088    91.51
1089    91.44
1090    91.57
1091    91.29
1092    91.24
1093    91.31
Name: schladen, dtype: float64

In [231]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["schladen"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=schladen, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["schladen"].min(), schladen.min()))
y_max = np.ceil(max(filtered_df["schladen"].max(), fill.max()))

fig.update_layout(
    title="Schladen Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 10, y_max + 10],
        dtick=5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [232]:
ohrum_weather = get_forecast("ohrum")
ohrum = predict_ohrum(schladen, ohrum_weather)
ohrum

array([77.3695 , 77.6229 , 77.1545 , 77.15334, 77.14864, 77.12674,
       76.8439 , 76.9951 , 77.1214 , 77.3264 ])

In [233]:
filtered_df = df_ohrum[(df_ohrum["timestamp"] >= "2023-12-21") & (df_ohrum["timestamp"] <= "2023-12-30")]
filtered_df["ohrum_level"]

1084    78.21
1085    78.92
1086    79.20
1087    79.56
1088    79.79
1089    79.87
1090    79.87
1091    79.83
1092    79.75
1093    79.71
Name: ohrum_level, dtype: float64

In [234]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["ohrum_level"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=ohrum, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["ohrum_level"].min(), schladen.min()))
y_max = np.ceil(max(filtered_df["ohrum_level"].max(), fill.max()))

fig.update_layout(
    title="Ohrum Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 10, y_max + 10],
        dtick=5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [235]:
# schaeferbruecke
schaeferbruecke_weather = get_forecast("schaeferbruecke")
schaeferbruecke = predict_bridge(ohrum, schaeferbruecke_weather)
schaeferbruecke

array([72.904 , 73.2162, 72.7962, 72.7591, 72.7714, 72.7644, 72.5492,
       72.5676, 72.7651, 72.8387])

In [236]:
filtered_df = df_bridge[(df_bridge["timestamp"] >= "2023-12-21") & (df_bridge["timestamp"] <= "2023-12-30")]
filtered_df["schaeferbridge"]

1084    73.79
1085    74.46
1086    74.83
1087    74.94
1088    75.17
1089    75.22
1090    75.22
1091    75.21
1092    75.15
1093    75.13
Name: schaeferbridge, dtype: float64

In [237]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["schaeferbridge"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=schaeferbruecke, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["schaeferbridge"].min(), schladen.min()))
y_max = np.ceil(max(filtered_df["schaeferbridge"].max(), fill.max()))

fig.update_layout(
    title="Bridge Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 10, y_max + 10],
        dtick=5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [238]:
eisenbuetteler_wehr_weather = get_forecast("eisenbuetteler_wehr")
eisenbuetteler_wehr = predict_eisenbuettel(schaeferbruecke, eisenbuetteler_wehr_weather)

# wendenwehr
wendenwehr_weather = get_forecast("wendenwehr")
wendenwehr = predict_wendenwehr(eisenbuetteler_wehr, wendenwehr_weather)

print(eisenbuetteler_wehr)
print(wendenwehr)

[70.6355 70.6657 70.5181 70.5499 70.6475 70.6178 70.7651 70.7441 70.7614
 70.7155]
[68.8478 68.7218 68.6    68.9646 68.5248 68.904  69.2925 69.2157 68.7884
 69.1154]


In [239]:
filtered_df = df_eisenbuettel[(df_eisenbuettel["timestamp"] >= "2023-12-21") & (df_eisenbuettel["timestamp"] <= "2023-12-30")]
filtered_df["eisenbuetteler_wehr"]

1084    70.40
1085    70.35
1086    70.15
1087    70.24
1088    70.54
1089    71.03
1090    71.06
1091    71.07
1092    70.72
1093    70.60
Name: eisenbuetteler_wehr, dtype: float64

In [240]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["eisenbuetteler_wehr"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=eisenbuetteler_wehr, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["eisenbuetteler_wehr"].min(), schladen.min()))
y_max = np.ceil(max(filtered_df["eisenbuetteler_wehr"].max(), fill.max()))

fig.update_layout(
    title="Eisenbuttel Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 10, y_max + 10],
        dtick=5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [241]:
forecast()

array([68.8478, 68.7218, 68.6   , 68.9646, 68.5248, 68.904 , 69.2925,
       69.2157, 68.7884, 69.1154])

In [242]:
filtered_df = df_wendenwehr[(df_wendenwehr["timestamp"] >= "2023-12-21") & (df_wendenwehr["timestamp"] <= "2023-12-30")]
filtered_df["wendenwehr"]

1084    69.03
1085    68.26
1086    68.04
1087    69.09
1088    68.23
1089    68.56
1090    68.75
1091    68.67
1092    68.42
1093    68.27
Name: wendenwehr, dtype: float64

In [243]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=filtered_df["wendenwehr"], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=filtered_df["timestamp"], y=wendenwehr, mode='lines', name='Forecast'))

y_min = np.floor(min(filtered_df["wendenwehr"].min(), wendenwehr.min()))
y_max = np.ceil(max(filtered_df["wendenwehr"].max(), wendenwehr.max()))

fig.update_layout(
    title="Wendenwehr Actual vs Forecast Data",
    xaxis=dict(
        tickangle=45
    ),
    yaxis=dict(
        range=[y_min - 0.5, y_max + 0.5],
        dtick=0.5
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()
